In [73]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import urllib
from dotenv import load_dotenv
import pandas as pd
from pprint import pprint
import requests
import os 
import base64
import json

In [74]:
load_dotenv()

SpotifyClientID = os.getenv("SPOTIPY_CLIENT_ID")
SpotifyClientSecret = os.getenv("SPOTIPY_CLIENT_SECRET")
SpotifyRedirectURI = os.getenv("SPOTIPY_REDIRECT_URI")
SpotifyOath = os.getenv("SpotifyOath")

def _make_authorization_headers(client_id, client_secret):
    auth_header = base64.b64encode(
        f"{client_id}:{client_secret}".encode("ascii")
    )
    return {"Authorization": "Basic %s" % auth_header.decode("ascii")}

baseauth = _make_authorization_headers(SpotifyClientID, SpotifyClientSecret)

tok = requests.post(r"https://accounts.spotify.com/api/token", 
                   data={"grant_type": "client_credentials"},
                   headers=baseauth)
btok = json.loads(tok.content)['access_token']
bear = {"Authorization": f"Bearer {btok}"}

In [3]:
# songlist = pd.read_csv("SongList.csv", encoding='latin1')
# songs = list(zip(songlist.Song.tolist(), songlist.Artist.tolist()))
# songs[0]

movie_list = pd.read_csv("1990_2020_movie_list1227.csv", encoding='latin1')
ml_ls = movie_list[["title", "movieid", "year"]].values.tolist()
ml_ls[:5]

[['9', 'tt0472033', 2009],
 ['21', 'tt0478087', 2008],
 ['42', 'tt0453562', 2013],
 ['54', 'tt0120577', 1998],
 ['300', 'tt0416449', 2007]]

In [11]:
acc_df = pd.DataFrame()

for movie, movieid, year in ml_ls:
    
#     track = "(If You're Wondering If I Want You To) I Want You To"
#     artist = "Weezer"
    #q=album:gold%20artist:abba&type=album

# movie = "2 Fast 2 Furious"
# year = "2007"
    yearp2 = int(year) + 2

    options = 'album,playlist'
    q = urllib.parse.quote(f"{movie} soundtrack year:{year}-{yearp2}")
    o = urllib.parse.quote(options)
    getme = f"https://api.spotify.com/v1/search?query={q}" \
        + f"&type={o}&market=US&offset=0&limit=1"
    output = requests.get(getme, headers=bear)
    jl = json.loads(output.content)
    alb = pd.json_normalize(jl['albums']['items'])
    alb['movieid'] = movieid
    ply = pd.json_normalize(jl['playlists']['items'])
    ply['movieid'] = movieid
    acc_df = acc_df.append(alb)
    acc_df = acc_df.append(ply)


# options = 'track'
# q = urllib.parse.quote(f"track:{track}") + r"%20" + urllib.parse.quote(f"artist:{artist}")
# o = urllib.parse.quote(options)
# getme = f"https://api.spotify.com/v1/search?query={q}" \
#     + f"&type={o}&market=US&offset=0&limit=10"
# output = requests.get(getme, headers=bear)
# jl = json.loads(output.content)
# ply = jl['tracks']['items']
# df = pd.json_normalize(ply)
# df['src_track'] = track
# df['src_artist'] = artist
# acc_df = acc_df.append(df.head(1))
acc_df.shape

(1574, 13)

In [12]:
acc_df.head()

,album_type,artists,href,id,images,name,release_date,release_date_precision,total_tracks,type,uri,external_urls.spotify,movieid
0,album,[{'external_urls': {'spotify': 'https://open.s...,https://api.spotify.com/v1/albums/687Jrlty7ZRF...,687Jrlty7ZRFyWEPvYNEHC,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Red vs. Blue Season 9 Soundtrack,2011-12-07,day,23.0,album,spotify:album:687Jrlty7ZRFyWEPvYNEHC,https://open.spotify.com/album/687Jrlty7ZRFyWE...,tt0472033
0,album,[{'external_urls': {'spotify': 'https://open.s...,https://api.spotify.com/v1/albums/76UcXpcqJHJQ...,76UcXpcqJHJQDoTkoAgWpq,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Celda 211 (Original Motion Picture Soundtrack),2010-11-16,day,12.0,album,spotify:album:76UcXpcqJHJQDoTkoAgWpq,https://open.spotify.com/album/76UcXpcqJHJQDoT...,tt0478087
0,compilation,[{'external_urls': {'spotify': 'https://open.s...,https://api.spotify.com/v1/albums/518q9gHXuXdE...,518q9gHXuXdE9AW8kaJjPC,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",42 (Original Motion Picture Soundtrack),2013-04-09,day,13.0,album,spotify:album:518q9gHXuXdE9AW8kaJjPC,https://open.spotify.com/album/518q9gHXuXdE9AW...,tt0453562
0,compilation,[{'external_urls': {'spotify': 'https://open.s...,https://api.spotify.com/v1/albums/2MhORIIiEX9g...,2MhORIIiEX9gE51YVlTzKO,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",300 Original Motion Picture Soundtrack (U.S. V...,2007-03-05,day,25.0,album,spotify:album:2MhORIIiEX9gE51YVlTzKO,https://open.spotify.com/album/2MhORIIiEX9gE51...,tt0416449
0,album,[{'external_urls': {'spotify': 'https://open.s...,https://api.spotify.com/v1/albums/51C0M5HRIAgF...,51C0M5HRIAgFyy5UhTEav7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1408 (Original Motion Picture Soundtrack),2007-01-01,day,16.0,album,spotify:album:51C0M5HRIAgFyy5UhTEav7,https://open.spotify.com/album/51C0M5HRIAgFyy5...,tt0450385


In [13]:
acc_df.to_csv("spotify_movie_soundtrack_search.csv")

In [4]:
movie_list = pd.read_csv("spotify_soundtrack_list.csv", encoding='latin1')
ml_ls = movie_list[["movieid", "uri"]].values.tolist()
ml_ls[:10]

[['tt6924650', 'spotify:album:0047X5dvfUO2PeWF3utR1Q'],
 ['tt0298148', 'spotify:album:009d83fu2guwouIwqr7i9E'],
 ['tt0363771', 'spotify:album:01wn5MrXTUWjdYTLYC2wqU'],
 ['tt0844471', 'spotify:album:02fC1zrM190lj46sEnBbVk'],
 ['tt0433387', 'spotify:album:02gcOu5jHffPVvKXapzUSq'],
 ['tt7131622', 'spotify:album:02GEKxoVe5ITAj68mZRAM7'],
 ['tt0491152', 'spotify:album:02Jk1LLUyRNSvdj2hr1eky'],
 ['tt2980516', 'spotify:album:02VRifrsiTM73hPGjXduRQ'],
 ['tt0387514', 'spotify:album:03ayxHF1nibS7OBP5OgAGU'],
 ['tt0479884', 'spotify:album:03JyRdIko8sx1jmnrMFd0r']]

In [22]:
acc_df = pd.DataFrame()

for movieid, uri in ml_ls:
# movieid = 'tt6924650'
# uri = 'spotify:playlist:2EKcKVdbLZ5hfNNSJPulcP'
    try:
        splt = uri.split(":")
        albid = splt[-1]
        options = splt[1]
        #     track = "(If You're Wondering If I Want You To) I Want You To"
        #     artist = "Weezer"
        #q=album:gold%20artist:abba&type=album
        #     q = urllib.parse.quote(f"{movie} soundtrack year:{year}-{yearp2}")
        #     o = urllib.parse.quote(options)

        #GET https://api.spotify.com/v1/albums/{id}/tracks
        #GET https://api.spotify.com/v1/playlists/{playlist_id}/tracks
        getme = f"https://api.spotify.com/v1/{options}s/{albid}"
        #https://api.spotify.com/v1/albums/05RzhgEEcsjQcFPJ4w7xSI
        output = requests.get(getme, headers=bear)
        jl = json.loads(output.content)

        # if options == "album":
        alb = pd.json_normalize(jl['tracks']['items'])
        alb['movieid'] = movieid
        alb['uri'] = uri
        acc_df = acc_df.append(alb)
    except:
        pass
#     acc_df
# elif options == "playlist":
#     ply = pd.json_normalize(jl['items'])
#     ply['movieid'] = movieid
#     acc_df = acc_df.append(ply)

acc_df.head()

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,preview_url,...,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,104666.0,False,https://api.spotify.com/v1/tracks/0HD1pOzeKONI...,0HD1pOzeKONIaHoJmfF7Hm,False,Nobody Wants A War,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,317826.0,False,https://api.spotify.com/v1/tracks/4D3bjstAjxLY...,4D3bjstAjxLYqTvfz3Z5Fe,False,Midway Main Theme,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,213920.0,False,https://api.spotify.com/v1/tracks/4SKVOC02Ilgv...,4SKVOC02IlgvT6Zt7b3ydX,False,See You In China,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,155693.0,False,https://api.spotify.com/v1/tracks/3DFzhIHyjhxR...,3DFzhIHyjhxRiXJOy6JVR9,False,Morgue,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1.0,155453.0,False,https://api.spotify.com/v1/tracks/27FitGAsI00y...,27FitGAsI00yMDUBah1Z1M,False,Getting Some Fun Out Of Life,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
acc_df.to_csv(r"soundtracklist.csv")

In [130]:
track_list = pd.read_csv("soundtracklist_coalesce.csv", encoding='latin1')
tls = track_list[["id", "uri", "movieid"]].values.tolist()
ids = track_list.id.tolist()
track_list.head()

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,preview_url,track_number,type,uri,external_urls.spotify,movieid
0,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,272160,False,https://api.spotify.com/v1/tracks/4TbSwQoXoVGR...,4TbSwQoXoVGRfLP0Uie3HX,False,Leonard - Awakenings - Original Motion Picture...,https://p.scdn.co/mp3-preview/bfac6ce5dc857ea8...,1,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/4TbSwQoXoVGRfLP...,tt0099077
1,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,99600,False,https://api.spotify.com/v1/tracks/3nPXi01v23gh...,3nPXi01v23ghfv1YNCOWr7,False,Dr. Sayer - Awakenings - Original Motion Pictu...,https://p.scdn.co/mp3-preview/2984bb3740645502...,2,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/3nPXi01v23ghfv1...,tt0099077
2,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,191440,False,https://api.spotify.com/v1/tracks/2oONzoMdgn27...,2oONzoMdgn27Hr1VXiDG9d,False,Lucy - Awakenings - Original Motion Picture So...,https://p.scdn.co/mp3-preview/b4249c94f5778c97...,3,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/2oONzoMdgn27Hr1...,tt0099077
3,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,70693,False,https://api.spotify.com/v1/tracks/72x2q9PDyqHL...,72x2q9PDyqHLZzUaJAaBHC,False,Catch - Awakenings - Original Motion Picture S...,https://p.scdn.co/mp3-preview/b171e4729fc6ace3...,4,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/72x2q9PDyqHLZzU...,tt0099077
4,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,191200,False,https://api.spotify.com/v1/tracks/4Qk4s7HmcwGR...,4Qk4s7HmcwGRb077Zw33bX,False,Rilke's Panther - Awakenings - Original Motion...,https://p.scdn.co/mp3-preview/bdbfaa95c28d6173...,5,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/4Qk4s7HmcwGRb07...,tt0099077


In [63]:
len(ids)

25781

In [64]:
def divide_chunks(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
  
ids_chunks = list(divide_chunks(ids, 49)) 

In [101]:
len(ids_chunks[0])

49

In [95]:
#GET https://api.spotify.com/v1/audio-features
#
# QUERY PARAMETER	TYPE	REQUIRED
# ids
# A comma-separated list of the Spotify IDs for the tracks. Maximum: 100 IDs.	String	Required
tracks = []
# ids_chunks = [['4TbSwQoXoVGRfLP0Uie3HX', '3nPXi01v23ghfv1YNCOWr7', '2oONzoMdgn27Hr1VXiDG9d', 
#                '72x2q9PDyqHLZzUaJAaBHC', '4Qk4s7HmcwGRb077Zw33bX', '3DUeuloGI7hFOTNJtp93LX', 
#                '6aU0ZlkNoUozpuc26pHIrN', '4KoPsMuKwcMSnWwnTcdOIk', '02GNLzJNm3PinZfWNlGYn8', 
#                '5YxU6GIF804YH3MG9UccfS', '395lgB4tU1Xrr8snl1U75z', '7dcQzzksdBi9vJLwtvwMKT', 
#                '2CPvpnaBXgdFCp1znjQpGT', '0zacyHAkDgmszIvUVSCfjS']]
for chunk in ids_chunks:
#chunk = ids_chunks[0]
    #try:
    comdelim = urllib.parse.quote(','.join([str(x) for x in chunk]))
    #print(comdelim)
    # #getme = f"https://api.spotify.com/v1/audio-features?ids={comdelim}" 
    getme = f"https://api.spotify.com/v1/tracks?ids={comdelim}" 
    output = requests.get(getme, headers=bear)
    jl = json.loads(output.text)['tracks']
    # #jl = json.loads(output.content)
    tracks += jl
    #     except:
#         pass

#tracks[0]['popularity']

In [119]:
tr_copy = tracks.copy()

for x in tr_copy:
    try:
        x.pop('album', None)
    except:
        pass
    try:
        x.pop('available_markets', None)
    except:
        pass

25781

In [126]:
tr_copy = [x for x in tr_copy if x]
print(len(tr_copy))
type(tr_copy[0])

25778


dict

In [131]:
import ast 

songfeatures = pd.json_normalize(tr_copy) #[["name", "popularity", "id", "external_ids"]]
print(len(songfeatures.index))
songfeatures.head()

25778


,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,preview_url,track_number,type,uri,external_ids.isrc,external_urls.spotify,artists
0,1,272160,False,https://api.spotify.com/v1/tracks/4TbSwQoXoVGR...,4TbSwQoXoVGRfLP0Uie3HX,False,Leonard - Awakenings - Original Motion Picture...,14,https://p.scdn.co/mp3-preview/bfac6ce5dc857ea8...,1,track,spotify:track:4TbSwQoXoVGRfLP0Uie3HX,USRE10800046,https://open.spotify.com/track/4TbSwQoXoVGRfLP...,NaN
1,1,99600,False,https://api.spotify.com/v1/tracks/3nPXi01v23gh...,3nPXi01v23ghfv1YNCOWr7,False,Dr. Sayer - Awakenings - Original Motion Pictu...,11,https://p.scdn.co/mp3-preview/2984bb3740645502...,2,track,spotify:track:3nPXi01v23ghfv1YNCOWr7,USRE10800045,https://open.spotify.com/track/3nPXi01v23ghfv1...,NaN
2,1,191440,False,https://api.spotify.com/v1/tracks/2oONzoMdgn27...,2oONzoMdgn27Hr1VXiDG9d,False,Lucy - Awakenings - Original Motion Picture So...,8,https://p.scdn.co/mp3-preview/b4249c94f5778c97...,3,track,spotify:track:2oONzoMdgn27Hr1VXiDG9d,USRE10800047,https://open.spotify.com/track/2oONzoMdgn27Hr1...,NaN
3,1,70693,False,https://api.spotify.com/v1/tracks/72x2q9PDyqHL...,72x2q9PDyqHLZzUaJAaBHC,False,Catch - Awakenings - Original Motion Picture S...,8,https://p.scdn.co/mp3-preview/b171e4729fc6ace3...,4,track,spotify:track:72x2q9PDyqHLZzUaJAaBHC,USRE10800048,https://open.spotify.com/track/72x2q9PDyqHLZzU...,NaN
4,1,191200,False,https://api.spotify.com/v1/tracks/4Qk4s7HmcwGR...,4Qk4s7HmcwGRb077Zw33bX,False,Rilke's Panther - Awakenings - Original Motion...,7,https://p.scdn.co/mp3-preview/bdbfaa95c28d6173...,5,track,spotify:track:4Qk4s7HmcwGRb077Zw33bX,USRE10800049,https://open.spotify.com/track/4Qk4s7HmcwGRb07...,NaN


In [134]:
comb = pd.merge(track_list, songfeatures[["name", "popularity", "id"]], how="inner", on="id")
# # comb['isrc_id'] = comb.external_ids.apply(lambda x: x['isrc'])
# # comb['artists'] = comb.artists.apply(lambda x: ast.literal_eval(x))
# # comb['artist_list'] = comb.artists.apply(lambda x: ",".join([y['name'] for y in x]))
# # comb['artist_ids'] = comb.artists.apply(lambda x: ",".join([y['id'] for y in x]))
# #comb = comb.drop(columns=['external_ids', 'available_markets', 'artists'])
#comb = comb.drop_duplicates()
# #comb.iloc[0,0]
comb.head()

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name_x,preview_url,track_number,type,uri,external_urls.spotify,movieid,name_y,popularity
0,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,272160,False,https://api.spotify.com/v1/tracks/4TbSwQoXoVGR...,4TbSwQoXoVGRfLP0Uie3HX,False,Leonard - Awakenings - Original Motion Picture...,https://p.scdn.co/mp3-preview/bfac6ce5dc857ea8...,1,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/4TbSwQoXoVGRfLP...,tt0099077,Leonard - Awakenings - Original Motion Picture...,14
1,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,99600,False,https://api.spotify.com/v1/tracks/3nPXi01v23gh...,3nPXi01v23ghfv1YNCOWr7,False,Dr. Sayer - Awakenings - Original Motion Pictu...,https://p.scdn.co/mp3-preview/2984bb3740645502...,2,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/3nPXi01v23ghfv1...,tt0099077,Dr. Sayer - Awakenings - Original Motion Pictu...,11
2,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,191440,False,https://api.spotify.com/v1/tracks/2oONzoMdgn27...,2oONzoMdgn27Hr1VXiDG9d,False,Lucy - Awakenings - Original Motion Picture So...,https://p.scdn.co/mp3-preview/b4249c94f5778c97...,3,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/2oONzoMdgn27Hr1...,tt0099077,Lucy - Awakenings - Original Motion Picture So...,8
3,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,70693,False,https://api.spotify.com/v1/tracks/72x2q9PDyqHL...,72x2q9PDyqHLZzUaJAaBHC,False,Catch - Awakenings - Original Motion Picture S...,https://p.scdn.co/mp3-preview/b171e4729fc6ace3...,4,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/72x2q9PDyqHLZzU...,tt0099077,Catch - Awakenings - Original Motion Picture S...,8
4,[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE...",1,191200,False,https://api.spotify.com/v1/tracks/4Qk4s7HmcwGR...,4Qk4s7HmcwGRb077Zw33bX,False,Rilke's Panther - Awakenings - Original Motion...,https://p.scdn.co/mp3-preview/bdbfaa95c28d6173...,5,track,spotify:album:40tpJ1Z4F6p8sJJ7fljb5R,https://open.spotify.com/track/4Qk4s7HmcwGRb07...,tt0099077,Rilke's Panther - Awakenings - Original Motion...,7


In [60]:
len(comb)

49

In [135]:
comb.to_csv("spotify_song_details.csv")